In [2]:
class EXP:
    MODEL = '/root/autodl-fs/hf-checkpoints/sentence-transformers/sentence-t5-base'
    # MODEL = '/root/autodl-fs/hf-checkpoints/BAAI/bge-base-en-v1.5'
    # MODEL = '/root/autodl-fs/hf-checkpoints/Alibaba-NLP/gte-base-en-v1.5'
    # MODEL = '/root/autodl-fs/hf-checkpoints/hkunlp/instructor-base'
    # MODEL = '/root/autodl-fs/hf-checkpoints/princeton-nlp/sup-simcse-bert-base-uncased'
    # MODEL = '/root/autodl-fs/hf-checkpoints/WhereIsAI/UAE-Large-V1'
    # MODEL = '/root/autodl-fs/hf-checkpoints/intfloat/e5-base'
    # MODEL = 'Alibaba-NLP/gte-base-en-v1.5'
    # MODEL = 'Alibaba-NLP/gte-large-en-v1.5'
    # MODEL = 'intfloat/e5-mistral-7b-instruct'
    # MODEL = 'BAAI/bge-base-en-v1.5'
    # MODEL = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
    # MODEL = "Alibaba-NLP/gte-Qwen2-7B-instruct"
    # MODEL = "princeton-nlp/sup-simcse-bert-large-uncased"
    # MODEL = "Salesforce/SFR-Embedding-2_R"
    # MODEL = "nomic-ai/nomic-embed-text-v1"
    # MODEL = "hkunlp/instructor-base"
    # MODEL = "hkunlp/instructor-xl"
    # MODEL = "WhereIsAI/UAE-Large-V1"
    # MODEL = "nomic-ai/nomic-bert-2048"
    # MODEL = "thenlper/gte-small"
    # MODEL = "Salesforce/SFR-Embedding-2_R"
    # MODEL = "GritLM/GritLM-7B"
    # MODEL = "sentence-transformers/gtr-t5-base"
    # MODEL = "nomic-ai/nomic-embed-text-v1.5"
    # MODEL = 'sentence-transformers/all-mpnet-base-v2'
    # MODEL  = "sentence-transformers/all-MiniLM-L6-v2"
    DATASET = [
        "mteb/sts13-sts",
        "mteb/sts22-crosslingual-sts",
        "mteb/sts12-sts",
        "mteb/stsbenchmark-sts",
        "mteb/sickr-sts",
        "mteb/sts14-sts",
        "mteb/biosses-sts",
        "mteb/sts16-sts",
        "mteb/sts15-sts",
        "mteb/stsb_multi_mt",
        "mteb/sts17-crosslingual-sts"
    ]
    SENT_PAIR_NUM = 5
    INSERT_NUM = 8
    VERIFICATION_SENT_PAIR_NUM = 250

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append('/root/StickyToken')
# from stickytoken.utils import *

In [ ]:
from stickytoken.embedding_model import ModelAnalyzer
moda = ModelAnalyzer(EXP.MODEL)

In [ ]:
from stickytoken.sentence_pair import SentencePair,Dataset
sent_pairs = SentencePair(EXP.DATASET,moda.model,moda.model_name)

In [6]:
import pandas as pd
sentence_pairs_df = pd.read_csv(sent_pairs.sentence_pairs_path)

In [ ]:
sentence_pairs_df = sentence_pairs_df[sentence_pairs_df['sentence2'].apply(lambda x: len(moda.tokenizer.tokenize(x)) >= 30 and len(moda.tokenizer.tokenize(x)) <= 100)]
sentence_pairs_df

In [ ]:
# 计算相似度范围
min_similarity = sentence_pairs_df['similarity'].min()
max_similarity = sentence_pairs_df['similarity'].max()
min_similarity,max_similarity

In [ ]:
# 从最小相似度开始，每隔0.01的similarity取一个样本采样
similarity_step = 0.01

# 进行采样
sampled_df = pd.DataFrame()
selected_pairs = set()
current_similarity = min_similarity
while current_similarity <= max_similarity:
    sorted_indices = (sentence_pairs_df['similarity'] - current_similarity).abs().argsort()
    for idx in sorted_indices:
        candidate_row = sentence_pairs_df.iloc[[idx]]
        candidate_pair = (candidate_row['sentence1'].values[0], candidate_row['sentence2'].values[0])
        if candidate_pair not in selected_pairs:
            sampled_df = pd.concat([sampled_df, candidate_row])
            selected_pairs.add(candidate_pair)
            break
    current_similarity += similarity_step

# 升序排序
sampled_df = sampled_df.sort_values('similarity', ascending=True)

# 统计总共使用了多少个不重复的单个句子
unique_sentences = set(sampled_df['sentence1']).union(set(sampled_df['sentence2']))
print(f"总共使用了 {len(unique_sentences)} 个不重复的句子。")

In [14]:
remained_df = sampled_df.copy()
sampled_df = sampled_df.sample(25)

In [ ]:
print((49947 in sampled_df.index))

In [ ]:
sampled_df.head()

In [ ]:
import random
random.seed(42)
# 从benign_tokens中随机选择一个token作为benigntoken
benig_ntoken = random.choice(moda.vocab)
benig_ntoken

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# token = '</s>'
token = 'lucrarea'
# token = benig_ntoken
add_num = 30  # 假设add_num为3，可以根据需要调整

from stickytoken.utils import *

def modify_text(text, token, num, mode='prefix'):
            """
            根据指定模式修改文本
            
            参数:
            text (str): 原始文本
            token (str): 要插入的token
            num (int): 插入次数
            mode (str): 修改模式，可选 'prefix'|'suffix'|'insert'
            
            返回:
            list: 修改后的文本列表
            """
            if mode == 'prefix':
                return [token * i + text for i in range(1, num + 1)]
            elif mode == 'suffix':
                return [text + token * i for i in range(1, num + 1)]
            elif mode == 'insert':
                modified_texts = []
                temp_text = text
                for _ in range(num):
                    new_text = random_insert(temp_text, token, 1)
                    temp_text = new_text
                    modified_texts.append(new_text)
                return modified_texts
            else:
                raise ValueError("Invalid mode. Choose from 'prefix', 'suffix', or 'insert'")

# 使用modify_text函数对sentence2列进行修改
sampled_df['sentence2_list_suffix'] = sampled_df['sentence2'].apply(lambda x: modify_text(x, token, add_num, mode='suffix'))
sampled_df['sentence2_list_prefix'] = sampled_df['sentence2'].apply(lambda x: modify_text(x, token, add_num, mode='prefix'))
sampled_df['sentence2_list_insert'] = sampled_df['sentence2'].apply(lambda x: modify_text(x, token, add_num, mode='insert'))

sampled_df.head()

In [13]:
# 计算每个sentence1和sentence2列表中每个句子的余弦相似度
def calculate_similarity(row, input_col='sentence2_list_suffix'):
    sentence1 = row['sentence1']
    sentence2_list = row[input_col]
    similarities = []
    for sentence2 in sentence2_list:
        embeddings = moda.model.encode([sentence1, sentence2])
        cosine_sim = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
        similarities.append(cosine_sim)
    return similarities

# 更新similarity列
sampled_df['similarity_list_suffix'] = sampled_df.apply(calculate_similarity, axis=1)
sampled_df['similarity_list_prefix'] = sampled_df.apply(calculate_similarity, input_col='sentence2_list_prefix', axis=1)
sampled_df['similarity_list_insert'] = sampled_df.apply(calculate_similarity, input_col='sentence2_list_insert', axis=1)

In [14]:
sampled_df['similarity_list'] = sampled_df["similarity_list_suffix"]

In [ ]:
sampled_df.head()

In [15]:
df = sampled_df.copy()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# plt.rcParams['font.family'] = 'sans-serif'
# 设置全局字体样式
plt.rcParams.update({
    'font.size': 14,
    'font.weight': 'normal',
    'axes.labelweight': 'normal',
    'axes.titleweight': 'normal',
})

# 准备绘图数据
all_similarity_values = [val for sublist in df['similarity_list'] for val in sublist]
first_similarity_values = [lst[0] for lst in df['similarity_list']]
final_similarity_values = [lst[-1] for lst in df['similarity_list']]

# 设置颜色映射
norm = plt.Normalize(min(first_similarity_values), 1)
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)

# 创建绘图
fig, axes = plt.subplots(1, 2, figsize=(9, 6), gridspec_kw={"width_ratios": [4, 0.5], "wspace": 0.1})
line_ax, box_ax = axes

# 折线图
for idx, row in df.iterrows():
    line_ax.plot(row['similarity_list'], color=sm.to_rgba(row['similarity_list'][0]), alpha=0.7)

line_ax.set_xlabel("Inserted number of sticky token", fontsize=16)
line_ax.set_ylabel("Cosine similarity", fontsize=16)
line_ax.grid(True, linestyle='--')
line_ax.set_xticks(range(0, len(df.iloc[0]['similarity_list']), 2))
line_ax.tick_params(axis='both', which='major', labelsize=14)

# 箱线图
sns.boxplot(y=final_similarity_values, ax=box_ax, color="skyblue", width=0.3, fliersize=0,  linewidth=0.8)
box_ax.set_xticklabels([])
box_ax.set_yticklabels([''] * len(box_ax.get_yticks()))
box_ax.grid(True, axis="y", linestyle='--')
box_ax.set_ylim(line_ax.get_ylim())
box_ax.tick_params(axis='both', which='major', labelsize=14)

# 添加颜色条并调小
# cbar = fig.colorbar(sm, cax=cbar_ax, orientation="vertical", label="Initial Similarity")
# cbar_ax.set_position([0.85, 0.38, 0.02, 0.5])
# cbar.ax.tick_params(labelsize=14)
# cbar.set_label("Initial Similarity", fontsize=16)

# 高质量保存
plt.savefig(r'/root/StickyToken/fig/sticky_symptoms_add.pdf', 
            format='pdf', 
            dpi=300, 
            bbox_inches='tight', 
            pad_inches=0.1)
plt.show()


# 生图函数

In [ ]:
sampled_df.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({
    'font.size': 14,
    'font.weight': 'normal',
    'axes.labelweight': 'normal',
    'axes.titleweight': 'normal',
})

def plot_similarity(token, sampled_df, add_num=40, similarity_list_col='similarity_list',
                    save_path='/root/StickyToken/fig/sticky_symptoms_add.pdf',
                    color = 'blue',
                    format='pdf',
                    font_size=22):
    """
    绘制相似度变化图
    :param token: 要添加的token
    :param sampled_df: 包含句子和相似度数据的DataFrame
    :param save_path: 图片保存路径
    """
    sampled_df = sampled_df.copy()[['sentence1', 'sentence2', 'similarity']]
    sampled_df['sentence2_list_suffix'] = sampled_df['sentence2'].apply(lambda x: [x + token * i for i in range(0, add_num + 1)])

    sampled_df[similarity_list_col] = sampled_df.apply(calculate_similarity, axis=1)
    # 准备绘图数据
    all_similarity_values = [val for sublist in sampled_df[similarity_list_col] for val in sublist]
    first_similarity_values = [lst[0] for lst in sampled_df[similarity_list_col]]
    final_similarity_values = [lst[-1] for lst in sampled_df[similarity_list_col]]

    # 设置颜色映射
    norm = plt.Normalize(min(first_similarity_values), 1)
    if color == 'blue':
        sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
    elif color == 'red':
        sm = plt.cm.ScalarMappable(cmap='magma', norm=norm)  # 使用黄红渐变色系

    # 创建绘图
    fig, axes = plt.subplots(1, 2, figsize=(9, 6), gridspec_kw={"width_ratios": [4, 0.5], "wspace": 0.1})
    line_ax, box_ax = axes

    # 创建颜色条轴
    cbar_ax = fig.add_axes([0.85, 0.38, 0.02, 0.5])

    # 折线图
    for idx, row in sampled_df.iterrows():
        line_ax.plot(row[similarity_list_col], color=sm.to_rgba(row[similarity_list_col][0]), alpha=0.7)

    if color == 'blue':
        line_ax.set_xlabel("Inserted number of random token", fontsize=font_size)
    elif color == 'red':
        line_ax.set_xlabel("Inserted number of sticky token", fontsize=font_size)
    line_ax.set_ylabel("Cosine similarity", fontsize=font_size)
    line_ax.grid(True, linestyle='--')
    line_ax.set_xticks(range(0, len(sampled_df.iloc[0][similarity_list_col]), 3))
    line_ax.set_xlim(0, len(sampled_df.iloc[0][similarity_list_col]))
    line_ax.tick_params(axis='both', which='major', labelsize=16)

    # 箱线图
    if color == 'blue':
        sns.boxplot(y=final_similarity_values, ax=box_ax, color="skyblue", width=0.3, fliersize=0, linewidth=0.8)
    elif color == 'red':
        sns.boxplot(y=final_similarity_values, ax=box_ax, color="#D6AFB9", width=0.3, fliersize=0, linewidth=0.8) 
    box_ax.set_xticklabels([])
    box_ax.set_yticklabels([''] * len(box_ax.get_yticks()))
    box_ax.grid(True, axis="y", linestyle='--')
    box_ax.set_ylim(line_ax.get_ylim())
    box_ax.tick_params(axis='both', which='major', labelsize=16)

    # 添加颜色条并调小，使用Reds颜色映射
    cbar = fig.colorbar(sm, cax=cbar_ax, orientation="vertical", label="Initial Similarity", cmap="Reds")
    cbar.ax.tick_params(labelsize=16)
    cbar.set_label("Initial Similarity", fontsize=font_size)
    
    # 设置整体图的标题
    # fig.suptitle(f'Add Token: {token}', fontsize=16, y=0.95)
    
    # 高质量保存
    plt.savefig(save_path, 
                format=format, 
                dpi=300, 
                bbox_inches='tight', 
                pad_inches=0.1)
    plt.show()

In [26]:
import pandas as pd
def get_sticky_token_list(model_name):
    sticky_tokens_of_all_models_df = pd.read_csv("../results/final_all_models_sticky_tokens.csv")
    # 先筛选出对应模型的行
    sticky_token_column = sticky_tokens_of_all_models_df[sticky_tokens_of_all_models_df['model'] == model_name]
    # 按照main_metric从高到低排序
    sticky_token_column = sticky_token_column.sort_values(by='main_metric', ascending=False)
    # 获取排序后的raw_vocab列表
    sticky_token_list = sticky_token_column['raw_vocab'].to_list()
    return sticky_token_list

In [ ]:
sticky_tokens = get_sticky_token_list(moda.model_name)
sticky_tokens

In [ ]:
for sticky_token in sticky_tokens:
    # 处理包含"/"的token，将其替换为"_"
    safe_token = sticky_token.replace("/", "_")
    save_path = f'/root/StickyToken/fig/sticky_symptoms_add/{moda.model_name}+{safe_token}.pdf'
    plot_similarity(sticky_token, sampled_df, add_num=30,similarity_list_col='similarity_list_suffix' ,save_path=save_path,color = 'red',format='png')


In [ ]:
moda.vocab

In [ ]:
import random
random.seed(42)
# 从benign_tokens中随机选择一个token作为benigntoken
benig_ntoken = random.choice(moda.vocab)
benig_ntoken = 'and'
print(benig_ntoken)
plot_similarity(benig_ntoken, sampled_df, add_num=30, save_path='/root/StickyToken/fig/benign_symptoms_add.pdf',color = 'blue',format='pdf')

In [ ]:
import random
from tqdm import tqdm
random.seed(42)
# 随机选择100个benign_token
benign_tokens = random.choices(moda.vocab, k=100)

for i, benign_token in tqdm(enumerate(benign_tokens), total=len(benign_tokens), desc="Processing benign tokens"):
    print(f"benign_token: {benign_token}")
    plot_similarity(benign_token, sampled_df, add_num=30, save_path=f'/root/StickyToken/fig/benign_symptoms_add/benign_symptoms_add_{i}.png')

# 替换

In [13]:
import random
random.seed(42)

def random_replace(text, replace_string, times):
    tokens = moda.tokenizer.tokenize(text)  # 使用tokenizer将句子分割成token列表
    for _ in range(times):
        replace_position = random.randint(0, len(tokens) - 1)  # 随机选择替换位置
        tokens[replace_position] = replace_string  # 在随机位置替换字符串
    return moda.tokenizer.convert_tokens_to_string(tokens)  # 将token列表重新组合成句子

# 使用random_replace函数处理'sentence2'列，嵌套调用random_replace，生成新的句子并保存到新的列'sentence2_replaced'
def nested_random_replace(text, replace_string, times):
    replaced_sentences = [text]
    for _ in range(times):
        text = random_replace(text, replace_string, 1)  # 每次调用random_replace时times=1
        replaced_sentences.append(text)
    return replaced_sentences

sampled_df['sentence2_replaced'] = sampled_df['sentence2'].apply(lambda x: nested_random_replace(x, token, add_num))

In [14]:
# 计算'sentence2_replaced'列和'sentence1'列的余弦相似度
def calculate_replaced_similarity(row):
    sentence1 = row['sentence1']
    sentence2_replaced_list = row['sentence2_replaced']
    similarities = []
    for sentence2_replaced in sentence2_replaced_list:
        embeddings = moda.model.encode([sentence1, sentence2_replaced])
        cosine_sim = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
        similarities.append(cosine_sim)
    return similarities

# 更新similarity_replaced列
sampled_df['similarity_replaced_list'] = sampled_df.apply(calculate_replaced_similarity, axis=1)


In [15]:
df = sampled_df.copy()

In [ ]:
all_similarity_values = [val for sublist in df['similarity_replaced_list'] for val in sublist]
first_similarity_values = [lst[0] for lst in df['similarity_replaced_list']]
final_similarity_values = [lst[-1] for lst in df['similarity_replaced_list']]

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
# 设置颜色映射
norm = plt.Normalize(min(first_similarity_values), 1)
sm = plt.cm.ScalarMappable(cmap='magma', norm=norm)  # 使用magma红黄色系

# 创建绘图
fig, axes = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={"width_ratios": [4, 0.5], "wspace": 0.1})
line_ax, box_ax = axes

# 折线图
for idx, row in df.iterrows():
    line_ax.plot(row['similarity_replaced_list'], color=sm.to_rgba(row['similarity_replaced_list'][0]), alpha=0.7)

line_ax.set_xlabel("Inserted number of sticky token", fontsize=18)
line_ax.set_ylabel("Cosine similarity", fontsize=18)
line_ax.grid(True, linestyle='--')
line_ax.set_xticks(range(0, len(df.iloc[0]['similarity_replaced_list']), 2))
line_ax.tick_params(axis='both', which='major', labelsize=14)

# 箱线图
sns.boxplot(y=final_similarity_values, ax=box_ax, color="#D6AFB9", width=0.3, fliersize=0, linewidth=0.8)  # 使用浅红色
box_ax.set_xticklabels([])
box_ax.set_yticklabels([''] * len(box_ax.get_yticks()))
box_ax.grid(True, axis="y", linestyle='--')
box_ax.set_ylim(line_ax.get_ylim())
box_ax.tick_params(axis='both', which='major', labelsize=14)

plt.show()
fig.savefig(r'/root/StickyToken/fig/symptoms_replace.pdf', format='pdf')